# Part 3. Deep Learning
## 4. 딥러닝 알고리즘
### 4.1 Dropout
* 과적합과 gradient vanishing 문제를 완화시킬 수 있는 알고리즘
* 신경망 학습 과정 중 레이어의 노드를 랜덤하게 drop 함으로써 generalization 효과를 가져오게 하는 테크닉
    * 즉 weight matrix에 랜덤하게 일부 column에 0을 집어넣어 연산
    * 레이어마다, 또 에포크마다 확률 값 다르게 지정

In [3]:
"""
모듈 임포트
"""
import numpy as np
import matplotlib.pyplot as plt
import torch

# 신경망 모듈 설계 시 필요한 함수 모아둔 모듈
import torch.nn as nn

# torch.nn 모듈 중에서도 자주 이용되는 함수 F로 지정
import torch.nn.functional as F
from torchvision import transforms, datasets

"""
장비 확인
"""
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

BATCH_SIZE = 32
EPOCHS = 10

"""
MNIST 데이터 다운로드
"""
train_dataset = datasets.MNIST(root = "../data/MNIST",
                              train = True,
                              download = True,
                              transform = transforms.ToTensor())
test_dataset = datasets.MNIST(root="../data/MNIST",
                             train = False,
                             transform = transforms.ToTensor())

# 다운로드 한 데이터셋을 미니배치 단위로 분리해 지정
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = BATCH_SIZE,
                                         shuffle = False)


In [18]:
"""
MLP 모델 설계
"""
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        
        self.dropout_prob = 0.5
        
    def forward(self, x):
        
        # view를 이용해 2차원 데이터를 1차원으로 변환(Flatten)
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        
        # 두번째 FF 레이어에 전달하기 위해 계산
        x = F.sigmoid(x)
        x = F.dropout(x, training = self.training, p = self.dropout_prob)
        x = self.fc2(x)
        
        x = F.sigmoid(x)
        x = F.dropout(x, training = self.training, p = self.dropout_prob)
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        
        return x


* training = self.training
    * 학습 상태일 때와 검증 상태일 때 따라 다르게 적용되기 위해 존재하는 파라미터
    * 학습할 때는 dropout을 사용하지만 검증 과정에서는 모든 노드를 사용해야하기 때문
    * model.train()으로 명시할 때 self.training = True / model.eval()로 명시할 때 self.training = False로 적용

In [19]:

"""
Optimizer, Objective Function 설정
"""

# 모델을 device에 할당
model = Net().to(DEVICE)

# back propagation을 이용해 파라미터 업데이트 시 이용하는 optimizer 정의
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

# loss 계산
criterion = nn.CrossEntropyLoss()

print(model)


Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)


In [22]:
"""
MLP 모델 학습을 진행해 학습 데이터에 대한 모델 성능 확인하는 함수 정의
"""
def train(model, train_loader, optimizer, log_interval):
    model.train()
    
    
    # train_loader에는 미니배치 단위로 학습에 이용되는 (image, label)이 저장되어 있음. 
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        
        # 각 파라미터에 할당된 gradient 값을 이용해 파라미터 값 업데이트
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{}({:.0f}%)] \t Train Loss: {:.6f}".format(Epoch, batch_idx * len(image),
                                                                                  len(train_loader.dataset), 
                                                                                  100. * batch_idx / len(train_loader), loss.item()))

"""
학습되는 과정 속에서 검증 데이터에 대한 모델 성능 확인하는 함수 정의
"""
def evaluate(model, test_loader):
    model.eval()

    test_loss = 0
    correct = 0
    
    # no_grad() 사용해 파라미터 업데이트 방지
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            
            test_loss += criterion(output, label).item()
            
            # 크기가 10인 벡터값 중 가장 큰 값에 대응하는 클래스로 예측했다고 판단
            prediction = output.max(1, keepdim = True)[1]
            
            # 올바르게 예측한 경우 count
            correct += prediction.eq(label.view_as(prediction)).sum().item()
            
    # 평균 loss 값 계산
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy



In [23]:
"""
학습 수행
"""
for Epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200, )
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:2f} %\n".format(Epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000(0%)] 	 Train Loss: 1.473844
Train Epoch: 1 [6400/60000(11%)] 	 Train Loss: 1.337460
Train Epoch: 1 [12800/60000(21%)] 	 Train Loss: 1.414527
Train Epoch: 1 [19200/60000(32%)] 	 Train Loss: 1.313331
Train Epoch: 1 [25600/60000(43%)] 	 Train Loss: 1.337585
Train Epoch: 1 [32000/60000(53%)] 	 Train Loss: 1.293303
Train Epoch: 1 [38400/60000(64%)] 	 Train Loss: 1.217575
Train Epoch: 1 [44800/60000(75%)] 	 Train Loss: 1.454268
Train Epoch: 1 [51200/60000(85%)] 	 Train Loss: 1.148197
Train Epoch: 1 [57600/60000(96%)] 	 Train Loss: 1.145716

[EPOCH: 1], 	Test Loss: 0.0286, 	Test Accuracy: 70.420000 %

Train Epoch: 2 [0/60000(0%)] 	 Train Loss: 0.979661
Train Epoch: 2 [6400/60000(11%)] 	 Train Loss: 1.136472
Train Epoch: 2 [12800/60000(21%)] 	 Train Loss: 0.924340
Train Epoch: 2 [19200/60000(32%)] 	 Train Loss: 1.016725
Train Epoch: 2 [25600/60000(43%)] 	 Train Loss: 1.111830
Train Epoch: 2 [32000/60000(53%)] 	 Train Loss: 1.212076
Train Epoch: 2 [38400/60000(64%)] 	 Tr

### 4.2 Activation 함수
#### 1. ReLU 함수
* 기존의 시그모이드 함수와 같은 비선형 활성 함수가 지니고 있는 문제점을 어느 정도 해결한 활성 함수
* f(x) = max(0, x)
* 입력 값이 0 이상인 부분은 기울기가 1, 0 이하인 부분은 0이 됨
    * 역전파 시 곱해지는 activation 미분값이 0 또는 1이 되기 때문에 아예 없애거나 완전히 살릴 수 있음
    * 이를 통해 hidden layer가 깊어져도 gradient vanishing이 일어나는 것을 완화시킬 수 있으며, 레이어를 깊게 쌓을 수 있음
* Leaky ReLU, ELU, parametric ReLU, SELU, SERLU 등 다양한 Activation 함수들이 파생

In [24]:
"""
MLP 모델 설계
"""
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        
        self.dropout_prob = 0.5
        
    def forward(self, x):
        
        # view를 이용해 2차원 데이터를 1차원으로 변환(Flatten)
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        
        # 두번째 FF 레이어에 전달하기 위해 계산
        x = F.relu(x)
        x = F.dropout(x, training = self.training, p = self.dropout_prob)
        x = self.fc2(x)
        
        x = F.relu(x)
        x = F.dropout(x, training = self.training, p = self.dropout_prob)
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        
        return x


In [26]:

"""
Optimizer, Objective Function 설정
"""

# 모델을 device에 할당
model = Net().to(DEVICE)

# back propagation을 이용해 파라미터 업데이트 시 이용하는 optimizer 정의
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

# loss 계산
criterion = nn.CrossEntropyLoss()

print(model)


Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)


In [27]:
"""
학습 수행
"""
for Epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200, )
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:2f} %\n".format(Epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000(0%)] 	 Train Loss: 2.296834
Train Epoch: 1 [6400/60000(11%)] 	 Train Loss: 2.012792
Train Epoch: 1 [12800/60000(21%)] 	 Train Loss: 1.237947
Train Epoch: 1 [19200/60000(32%)] 	 Train Loss: 0.846841
Train Epoch: 1 [25600/60000(43%)] 	 Train Loss: 0.814991
Train Epoch: 1 [32000/60000(53%)] 	 Train Loss: 0.767190
Train Epoch: 1 [38400/60000(64%)] 	 Train Loss: 0.312187
Train Epoch: 1 [44800/60000(75%)] 	 Train Loss: 0.369823
Train Epoch: 1 [51200/60000(85%)] 	 Train Loss: 0.681173
Train Epoch: 1 [57600/60000(96%)] 	 Train Loss: 0.296593

[EPOCH: 1], 	Test Loss: 0.0101, 	Test Accuracy: 90.890000 %

Train Epoch: 2 [0/60000(0%)] 	 Train Loss: 0.458644
Train Epoch: 2 [6400/60000(11%)] 	 Train Loss: 0.360178
Train Epoch: 2 [12800/60000(21%)] 	 Train Loss: 0.325507
Train Epoch: 2 [19200/60000(32%)] 	 Train Loss: 0.296544
Train Epoch: 2 [25600/60000(43%)] 	 Train Loss: 0.396078
Train Epoch: 2 [32000/60000(53%)] 	 Train Loss: 0.240016
Train Epoch: 2 [38400/60000(64%)] 	 Tr

### 4.3 Batch Normalization
* Internal Covarriance Shift: 각 레이어마다 Input 분포가 달라짐에 따라 학습 속도가 느려지는 현상
* Batch Normalization은 이를 방지하기 위한 기법으로 레이어의 Input 분포를 정규화 해 학습 속도를 빠르게 하는 것
* 수식: <img src="https://latex.codecogs.com/gif.latex?BN%28h%3B%5Cgamma%3B%5Cbeta%29%20%3D%20%5Cbeta%20&plus;%20%5Cgamma%20%5Cfrac%7Bh%20-%20E%28h%29%7D%7B%5Csqrt%7BVar%28h%29&plus;%5Cepsilon%7D%7D"/>
    * h: input의 분포
    * beta & gamma: 각각 분포를 shift시키고 scaling 시키는 값으로 역전파를 통해 학습
* BN을 통해 분포를 정규화 해 비선형 활성ㅇ 함수의 의미를 살리게 된다!
* 차원에 따라 적용되는 함수명이 다르기 때문에 유의해서 사용해야 한다.

In [28]:
"""
MLP 모델 설계
"""
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        
        self.dropout_prob = 0.5
        
        self.batch_norm1 = nn.BatchNorm1d(512)
        self.batch_norm2 = nn.BatchNorm1d(256)
        
    def forward(self, x):
        
        # view를 이용해 2차원 데이터를 1차원으로 변환(Flatten)
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        x = self.batch_norm1(x)
        
        # 두번째 FF 레이어에 전달하기 위해 계산
        x = F.relu(x)
        x = F.dropout(x, training = self.training, p = self.dropout_prob)
        x = self.fc2(x)
        x = self.batch_norm2(x)
        
        x = F.relu(x)
        x = F.dropout(x, training = self.training, p = self.dropout_prob)
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        
        return x


In [29]:

"""
Optimizer, Objective Function 설정
"""

# 모델을 device에 할당
model = Net().to(DEVICE)

# back propagation을 이용해 파라미터 업데이트 시 이용하는 optimizer 정의
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

# loss 계산
criterion = nn.CrossEntropyLoss()

print(model)


Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
  (batch_norm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [30]:
"""
학습 수행
"""
for Epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200, )
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:2f} %\n".format(Epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000(0%)] 	 Train Loss: 2.470002
Train Epoch: 1 [6400/60000(11%)] 	 Train Loss: 0.412125
Train Epoch: 1 [12800/60000(21%)] 	 Train Loss: 0.648231
Train Epoch: 1 [19200/60000(32%)] 	 Train Loss: 0.325135
Train Epoch: 1 [25600/60000(43%)] 	 Train Loss: 0.417800
Train Epoch: 1 [32000/60000(53%)] 	 Train Loss: 0.317921
Train Epoch: 1 [38400/60000(64%)] 	 Train Loss: 0.572682
Train Epoch: 1 [44800/60000(75%)] 	 Train Loss: 0.460256
Train Epoch: 1 [51200/60000(85%)] 	 Train Loss: 0.363026
Train Epoch: 1 [57600/60000(96%)] 	 Train Loss: 0.073160

[EPOCH: 1], 	Test Loss: 0.0047, 	Test Accuracy: 95.560000 %

Train Epoch: 2 [0/60000(0%)] 	 Train Loss: 0.339551
Train Epoch: 2 [6400/60000(11%)] 	 Train Loss: 0.129543
Train Epoch: 2 [12800/60000(21%)] 	 Train Loss: 0.154798
Train Epoch: 2 [19200/60000(32%)] 	 Train Loss: 0.317309
Train Epoch: 2 [25600/60000(43%)] 	 Train Loss: 0.320206
Train Epoch: 2 [32000/60000(53%)] 	 Train Loss: 0.136236
Train Epoch: 2 [38400/60000(64%)] 	 Tr

### 4. Initialization
* 신경망을 어떻게 초기화하냐에 따라 학습 속도가 달라진다.
* 종류:
    * LeCun Initialization
        * LeCun Normal Initialization
        * LeCun Uniform Initialization
    * Xavier Initialization
        * 이전 레이어의 노드 수와 다음 레이어의 노드 수에 따라 가중치를 결정
    * He Initialization
        * Xavier Initialization은 ReLU를 사용할 때 비효율적인데 이를 보완한 기법

* pytorch 내의 nn.linear는 어떤 분포에서 샘플링을 통해 파라미터를 초기화 할까?
    * output으로 계산되는 벡터의 차원 수의 역수 값에 대한 +/- 범위 내 uniform distribution을 설정해 샘플링
* 예제에서는 He Initialization을 이용해 파라미터를 초기화 해보자.

In [33]:
import torch.nn.init as init
def weight_init(m):
    if isinstance(m, nn.Linear): # 레이어 중 nn.Linear인 것에 대해서만 지정
        init.kaiming_uniform_(m.weight.data) # he_initialization을 이용해 파라미터 값 초기화
        
model = Net().to(DEVICE)
model.apply(weight_init)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)
criterion = nn.CrossEntropyLoss()

In [34]:
"""
학습 수행
"""
for Epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200, )
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:2f} %\n".format(Epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000(0%)] 	 Train Loss: 3.233096
Train Epoch: 1 [6400/60000(11%)] 	 Train Loss: 1.050164
Train Epoch: 1 [12800/60000(21%)] 	 Train Loss: 0.470191
Train Epoch: 1 [19200/60000(32%)] 	 Train Loss: 0.518469
Train Epoch: 1 [25600/60000(43%)] 	 Train Loss: 0.489643
Train Epoch: 1 [32000/60000(53%)] 	 Train Loss: 0.755374
Train Epoch: 1 [38400/60000(64%)] 	 Train Loss: 0.572384
Train Epoch: 1 [44800/60000(75%)] 	 Train Loss: 0.570972
Train Epoch: 1 [51200/60000(85%)] 	 Train Loss: 0.371977
Train Epoch: 1 [57600/60000(96%)] 	 Train Loss: 0.249508

[EPOCH: 1], 	Test Loss: 0.0069, 	Test Accuracy: 93.400000 %

Train Epoch: 2 [0/60000(0%)] 	 Train Loss: 0.700280
Train Epoch: 2 [6400/60000(11%)] 	 Train Loss: 0.490792
Train Epoch: 2 [12800/60000(21%)] 	 Train Loss: 0.320310
Train Epoch: 2 [19200/60000(32%)] 	 Train Loss: 0.201973
Train Epoch: 2 [25600/60000(43%)] 	 Train Loss: 0.237975
Train Epoch: 2 [32000/60000(53%)] 	 Train Loss: 0.244611
Train Epoch: 2 [38400/60000(64%)] 	 Tr